<a href="https://colab.research.google.com/github/Kwangbin/Keras_Basic/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/MyDrive/ml/data/titanic/

/content/drive/MyDrive/ml/data/titanic


In [2]:
ls

brooklyn.csv           president_heights.csv  train.csv
gender_submission.csv  queens.csv             weight-height01.csv
heights.csv            streeteasy.csv         weight-height02.csv
manhattan.csv          test.csv


In [3]:
import pandas as pd
passengers = pd.read_csv("train.csv")
print(passengers.shape)
print(passengers.head())

(891, 12)
   PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]


Survived가 살았는지 죽었는지 확인하는 레이블이 될 거다. 0 또는 1인데, 1이 살아남은 거다.

2. 데이터 전처리하기
분석에 사용할 feature 고르기
일단 생존 여부에 중요한 영향을 미쳤을 것으로 예상하는 값들만 추려보면, Sex, Age, Pcalss를 생각해볼 수 있다. 여성, 어린이, 1/2/3등석 순으로 살아남을 확률이 높다고 가설을 세워본 거다.

문자열을 숫자로 변환하기

일단 머신러닝 라이브러리 sklearn에 넣어주기 전에 성별(Sex)는 숫자 데이터 1과 0으로 바꿔주자. 여성이 살아남을 확률이 높을 것 같으니 남성을 0, 여성을 1로 놓자.

In [4]:
passengers['Sex'] = passengers['Sex'].map({'female':1,'male':0})

결측치 확인 하기

In [5]:
passengers.isnull()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,True,False
887,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,False,False,True,False,False,False,False,True,False
889,False,False,False,False,False,False,False,False,False,False,False,False


결측치 채워주기

데이터를 보면 Age 값이 비어있는 경우가 있다. 이 경우 그냥 평균 값으로 대체해서 넣어주자.

In [6]:
passengers['Age'].fillna(value=passengers['Age'].mean(), inplace=True)

feature 분리하기

Pcalss의 경우 1등석에 탔는지, 2등석에 탔는지에 대해 각각의 feature로 만들어주기 위해 컬럼을 새로 생성해보자.

In [7]:
passengers['FirstClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 1 else 0)
passengers['SecondClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 2 else 0)

데이터 세트를 이렇게 준비

In [8]:
features = passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survival = passengers['Survived']

3. 학습세트/평가세트 분리하기

sklearn의 train_test_split을 사용해서 학습세트와 평가세트를 분리하자.

In [9]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, survival)

4. 데이터 정규화(스케일링) 하기 ***

중요한 파트다. 로지스틱 회귀는 실행할 때 regularation을 사용하기 때문에 그 전에 반드시 우리가 가진 데이터를 스케일링하는 작업이 필요하다. (이 부분은 이해 안 가더라도 일단 그냥 해야 된다고 생각하자.)

sklearn이 제공하는 StandardScaler를 활용해서 손쉽게 할 수 있다. StandardScaler는 평균 0, 표준편차 1로 변환하는 방법이지만 이외에도 최소값 0, 최대값 1이 되도록 변환하는 MinMaxScaler, 중앙값(median) 0, IQR(interquartile range) 1이 되도록 변환하는 RobustScaler 등이 있다.

StandardScaler를 사용하면 아래와 같이 데이터를 정규화(스케일링) 할 수 있다.

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

fit_transform 은 fit과 transform을 합친 건데, fit은 일단 각 속성(feature)마다 컬럼을 만드는 작업이라고 생각하면 된다. 이후 transform을 통해 데이터를 변형시키는 거다.

그래서 위 코드에서도 학습세트로 fit을 한 번 해주었기 때문에, 평가세트에서는 별도로 fit을 할 필요 없이 바로 transform하면 되는 거다.

5. 모델 생성 및 평가하기

모델을 생성하는 방법은 아래와 같다.

데이터만 넣어주고 fit 해주면 끝이다.

In [11]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

그러면 이제 학습 세트로 정확도를 바로 찍어볼 수 있다.

In [12]:
print(model.score(train_features, train_labels))

0.8008982035928144


80% 정도 맞춘다는 것

아까 분리해놓은 테스트 세트에서도 확인해볼 수 있다.

In [13]:
print(model.score(test_features, test_labels))

0.7892376681614349


78% 정도 맞춘다는 것

계수(coefficients)를 확인

In [14]:
print(model.coef_)

[[ 1.2570793  -0.49106712  1.03566764  0.46623006]]


아까 Sex, Age, FirstClass, SecondClass 순으로 넣었기 때문에 그 순서대로 확인해주면 된다. 성별이 1(여자)이고, 일등석 탑승 여부가 중요하다는 걸 알 수 있다. 반면 나이에 대한 계수는 음수가 나오는데 나이가 많을수록 생존 확률이 낮아진다는 의미로 이해하면 되겠다.

5. 예측하기

이번엔 새로운 데이터를 넣어서 예측해보자.

예를 들면 아래와 같이 타이타닉 영화 실제 주인공이었던 Jack, Rose의 값을 임의로 만들고, 내 정보도 넣어봤다. (가상이니까… 1등석에 탄 걸로 해봤다.)

In [15]:
import numpy as np

Jack = np.array([0.0, 20.0, 0.0, 0.0])
Rose = np.array([1.0, 17.0, 1.0, 0.0])
ME = np.array([0.0, 32.0, 1.0, 0.0])

sample_passengers = np.array([Jack, Rose, ME])

그 다음 위에서 했던 것처럼 데이터 스케일링을 다시 해줘야 한다.

In [16]:
sample_passengers = scaler.transform(sample_passengers)

예측

In [20]:
print(model.predict(sample_passengers))

[0 1 1]


Jack  사망
Rose , Me  생존

확률을 확인해보자.

In [19]:
print(model.predict_proba(sample_passengers))

[[0.99453213 0.00546787]
 [0.00282303 0.99717697]
 [0.41243705 0.58756295]]
